### CS690
### HW1 
### Rui Huang 

In [83]:
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.charts import Histogram, HeatMap
from bokeh.layouts import row, column, gridplot, layout, widgetbox
from bokeh.models import CustomJS, ColumnDataSource, Slider, HoverTool, LinearAxis, CategoricalColorMapper
from bokeh.models.widgets import Select, Panel, Tabs, Div
import pandas as pd 
import math 
import numpy as np 
import csv
from collections import defaultdict
from ipywidgets import interact
from bokeh.core.properties import field
from bokeh.palettes import Spectral6 

output_notebook() 

# create a dict: country -> region 
region_dict = defaultdict(str)
with open("gapminder_regions.csv",'rb') as regions:
    dr = csv.DictReader(regions)
    for line in dr:
        region_dict[line['Country']] = line['Group']
        

# preprocess data for line graph 
data = pd.read_csv("ppl-living-with-hiv-all-ages.csv")
data.fillna(0, inplace=True)
line_years = data.columns.values[1:]

i_subSaharanAfrica = []
i_southAsia = []
i_middleEastNorthAfrica = []
i_america = []
i_europeCentralAsia = []
i_eastAsiaPacific = [] 

for i in range(len(data["People living with HIV"])): 
    if (region_dict[data["People living with HIV"].iloc[i]] == "Sub-Saharan Africa"): 
        i_subSaharanAfrica.append(i)
    elif (region_dict[data["People living with HIV"].iloc[i]] == "South Asia"): 
        i_southAsia.append(i)
    elif (region_dict[data["People living with HIV"].iloc[i]] == "Middle East & North Africa"):
        i_middleEastNorthAfrica.append(i)
    elif (region_dict[data["People living with HIV"].iloc[i]] == "America"):
        i_america.append(i)
    elif (region_dict[data["People living with HIV"].iloc[i]] == "Europe & Central Asia"):
        i_europeCentralAsia.append(i)
    elif (region_dict[data["People living with HIV"].iloc[i]] == "East Asia & Pacific"):
        i_eastAsiaPacific.append(i) 
        
sum_subSaharanAfrica = data.iloc[i_subSaharanAfrica].sum(axis=0).tolist()[1:]
sum_southAsia = data.iloc[i_southAsia].sum(axis=0).tolist()[1:]
sum_middleEastNorthAfrica = data.iloc[i_middleEastNorthAfrica].sum(axis=0).tolist()[1:]
sum_america = data.iloc[i_america].sum(axis=0).tolist()[1:]
sum_europeCentralAsia = data.iloc[i_europeCentralAsia].sum(axis=0).tolist()[1:]
sum_eastAsiaPacific = data.iloc[i_eastAsiaPacific].sum(axis=0).tolist()[1:]

line_source = {"Sub-Saharan Africa": sum_subSaharanAfrica, "South Asia": sum_southAsia, 
               "Middle East & North Africa": sum_middleEastNorthAfrica, "America": sum_america, 
              "Europe & Central Asia": sum_europeCentralAsia, "East Asia & Pacific": sum_eastAsiaPacific} 


Loading BokehJS ...

In [84]:
line_y = [sum_subSaharanAfrica, sum_southAsia, sum_middleEastNorthAfrica, sum_america, 
          sum_europeCentralAsia, sum_eastAsiaPacific] 
line_graph = figure(title="People living with HIV (all ages)", plot_height=400, plot_width=600)
line_colors = ["navy", "pink", "green", "yellow", "orange", "red"] 
line_legends = ["Sub-Saharan Africa", "South Asia", "Middle East & North Africa", "America", 
                "Europe & Central Asia", "East Asia & Pacific"] 

for y, line_color, legend_name in zip(line_y, line_colors, line_legends): 
    line_graph.line(line_years, y, color=line_color, legend=legend_name, line_width=3, 
                    muted_color=line_color, muted_alpha=0.2)

#line_r = line_graph.line(line_years, line_source["America"], color="#2222aa", line_width=3)
hover_line = HoverTool(
		tooltips=[
			("year", "$x"),
			("number", "$y"),
		])

line_graph.add_tools(hover_line) 
line_graph.legend.click_policy='mute'
line_graph.legend.location = "top_left"

# def update_line(region):
#     line_r.data_source.data['y'] = line_source[region]
#     push_notebook()




**This is an interactive line graph. Clicking legends can mute lines.** 

In [85]:
show(line_graph) 

In [86]:
# interact(update_line, region=["America", "Sub-Saharan Africa", "South Asia", "Middle East & North Africa", 
#                               "Europe & Central Asia", "East Asia & Pacific"])

In [87]:
num_ppl = pd.read_csv("ppl-living-with-hiv-all-ages.csv")
annual_death = pd.read_csv("annual-number-of-AIDS-deaths.csv")
num_ppl.fillna(0, inplace=True)
annual_death.fillna(0, inplace=True) 

In [88]:
i_need = []
for i in range(len(annual_death["Annual number of AIDS deaths"])): 
    if (region_dict[annual_death["Annual number of AIDS deaths"].iloc[i]]): 
        i_need.append(i) 
        
annual_death = annual_death.iloc[i_need]

In [89]:
def get_data(year): 
    tmp_annual_death = annual_death[annual_death[year]<annual_death[year].median()*100] 
    y = annual_death[year][annual_death[year]<annual_death[year].median()*100].tolist()
    #print len(y)
    annual_death_ctys = tmp_annual_death["Annual number of AIDS deaths"].tolist() 
    x = num_ppl[num_ppl["People living with HIV"].isin(annual_death_ctys)][year]
    #print len(x) 
    
    colors = [] 
    scatter_legends = []
    for c in annual_death_ctys: 
        if (region_dict[c] == "Sub-Saharan Africa"): 
            colors.append("navy")
            scatter_legends.append("Sub-Saharan Africa")
        elif (region_dict[c] == "South Asia"): 
            colors.append("pink")
            scatter_legends.append("South Asia")
        elif (region_dict[c] == "Middle East & North Africa"):
            colors.append("green")
            scatter_legends.append("Middle East & North Africa")
        elif (region_dict[c] == "America"):
            colors.append("yellow")
            scatter_legends.append("America")
        elif (region_dict[c] == "Europe & Central Asia"):
            colors.append("orange")
            scatter_legends.append("Europe & Central Asia")
        elif (region_dict[c] == "East Asia & Pacific"):
            colors.append("red")
            scatter_legends.append("East Asia & Pacific")
    return x, y, colors, scatter_legends
 

In [90]:
scatter_years = annual_death.columns.values[1:]
#print scatter_years 

In [94]:
scatter_x, scatter_y, scatter_colors, scatter_legends = get_data('1990')

#scatter_source = {'x': scatter_x, 'y': scatter_y, 'legend_names': scatter_legends} 
scatter_graph = figure(title="People living with HIV (all ages) vs. Annual death of AIDS", 
                       plot_height=500, plot_width=700, x_axis_label="People living with HIV (all ages)",
                      y_axis_label="Annual death of AIDS")
scatter_column_source = ColumnDataSource(dict(x=scatter_x, y=scatter_y, legend_names=scatter_legends))
# for x_i, y_i, color_i, legend_i in zip(scatter_x, scatter_y, scatter_colors, scatter_legends): 
#     scatter_graph.circle(x_i, y_i, size=10, fill_color=color_i, fill_alpha=0.6, line_color=None, legend=legend_i)

color_mapper = CategoricalColorMapper(palette=["navy", "pink", "green", "yellow", "orange", "red"], factors=["Sub-Saharan Africa", "South Asia", "Middle East & North Africa", "America", 
                "Europe & Central Asia", "East Asia & Pacific"]) 

scatter_r = scatter_graph.circle('x', 'y', size=10, source=scatter_column_source, 
                                 fill_color={'field': 'legend_names', 'transform': color_mapper}, 
                                 fill_alpha=0.6, line_color=None, legend='legend_names')

hover_scatter = HoverTool(
		tooltips=[
			("People living with HIV", "$x"),
			("Annual death of AIDS", "$y"),
		])


scatter_graph.add_tools(hover_scatter)

# def update_scatter(year=1990):
#     result = get_data(str(year)) 
#     scatter_r.data_source.data['x'] = result[0]
#     scatter_r.data_source.data['y'] = result[1] 
#     scatter_r.data_source.data['fill_color'] = result[2] 
#     #scatter_r.data_source.data['legend'] = result[3]
#     push_notebook()



# import warnings
# warnings.filterwarnings('ignore') 

** This scatter plot shows the distribution according to regions by different colors. 
It supports pan, wheel zoom, box zoom and hover. **

In [95]:
show(scatter_graph)

In [92]:
#interact(update_scatter, year=(1990, 2011))